<a href="https://colab.research.google.com/github/chriskary1/energy-forecasting-prophet/blob/main/notebooks/energy_forecast_prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from prophet import Prophet


!pip install gdown
import gdown

#
file_id = '1FFqfEseizl0nukORBYsS2M297edkm3cw'
url = f'https://drive.google.com/uc?id={file_id}'

output = 'AEP_hourly_1.csv'
gdown.download(url, output, quiet=False)


In [10]:
# read dataframe
df = pd.read_csv('AEP_hourly_1.csv')

#
df['ds'] = pd.to_datetime(df['ds'])
df = df.set_index('ds').resample('D').sum().reset_index()
df.rename(columns={'value': 'y'}, inplace=True)


In [11]:
df.describe()


,ds,y
count,2041,2041.000000
mean,2015-10-18 00:00:00.000000256,358394.758942
min,2013-01-01 00:00:00,14809.000000
25%,2014-05-26 00:00:00,323874.000000
50%,2015-10-18 00:00:00,352558.000000
75%,2017-03-11 00:00:00,390355.000000
max,2018-08-03 00:00:00,535660.000000
std,NaN,46524.265536


In [12]:
# model with holidays
m = Prophet()
m.add_country_holidays(country_name='US')

# Fit & Forecast
m.fit(df)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)

# view forcast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp3qz9a8z0/1neotcv1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp3qz9a8z0/h4kqp90a.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=51198', 'data', 'file=/tmp/tmp3qz9a8z0/1neotcv1.json', 'init=/tmp/tmp3qz9a8z0/h4kqp90a.json', 'output', 'file=/tmp/tmp3qz9a8z0/prophet_modeljxi2bmsn/prophet_model-20250806161413.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:14:13 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:14:13 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


             ds           yhat     yhat_lower     yhat_upper
2401 2019-07-30  401235.362881  361950.835289  441035.871936
2402 2019-07-31  399501.243575  360439.053759  440438.448002
2403 2019-08-01  399985.581002  360730.836951  442214.564010
2404 2019-08-02  391382.194083  350558.756627  430908.257819
2405 2019-08-03  364355.331518  322722.019847  404941.465879


In [13]:
from prophet.plot import plot_plotly, plot_components_plotly
plot_plotly(m, forecast)



In [14]:
plot_components_plotly(m, forecast)

/usr/local/lib/python3.11/dist-packages/plotly/io/_json.py:560: UserWarning:

Discarding nonzero nanoseconds in conversion.



In [15]:
# Εξαγωγή πλήρους forecast σε CSV
forecast.to_csv('forecast_results.csv', index=False)

# Λήψη του αρχείου τοπικά (Google Colab)
from google.colab import files
files.download('forecast_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>